In [214]:
import math
import logging
import pandas as pd
import netsquid as ns
import numpy as np
from tqdm import tqdm
from netsquid.components.models.qerrormodels import QuantumErrorModel
import netsquid.qubits.qubitapi as qapi
from netsquid.components.clock import Clock
from netsquid.qubits.ketstates import b00, b01, b10, b11, BellIndex
from netsquid.util import DataCollector
from netsquid.util.simlog import logger
from netsquid.util.simtools import get_random_state
from netsquid_ae.datacollectors import ChainStateCollector
from netsquid_ae.ae_chain_setup import create_repeater_chain, create_single_repeater,  create_qkd_application, create_elementary_link
from netsquid_ae.ae_classes import EndNode, RepeaterNode
from netsquid_ae.protocol_event_types import EVTYPE_SUCCESS
from netsquid.qubits.qubit import Qubit
from netsquid_ae.ae_protocols import ExtractionProtocol
from netsquid_ae.ae_classes import QKDNode



In [4]:
import os
import time
import logging
import pickle5 as pickle
from matplotlib import rcParams
import matplotlib.pyplot as plt
from argparse import ArgumentParser

import netsquid as ns
from netsquid.util import DataCollector
from netsquid.util.simlog import logger

from pydynaa import EventExpression, ExpressionHandler

from netsquid_ae.datacollectors import DecisionMaker, ChainStateCollector, MeasurementCollector


In [ ]:
sim_params = {
            # SIMULATION
            "multi_photon": False,          # bool: whether to use multi-photon up to n=3 (WARNING: very slow)
            "num_repeaters": 2,             # number of repeaters
            "encoding": "time_bin",         # encoding of the entangled photons
            # end of simulation             Note: only use one and set others to -1
            "num_attempts": 10,             # number of clicks after which the clock stops (thus stopping the protocols)
            "num_attempts_proto": -1,       # number of clicks after which the emission protocols stop
            "num_successes": -1,            # number of successes after which to stop the simulation
            # magic
            "magic": None,                  # whether we want to use magic (should be "analytical", "sampled" or None)
            "state_files_directory": '.',
            # clock                         Note: only used if magic = None, otherwise cycle time is used as time_step
            "time_step": 1e6,               # time step for the clock [ns]
            "multiple_link_successes": False,  # whether elementary links can have multiple successful modes

            # COMPONENTS
            # channel
            "length": 1,                    # total distance between end node/ total channel length [km]
            "channel_length_l": -1,         # channel length left of the detectors [km]
            "channel_length_r": -1,         # channel length right of the detectors [km]
            "coupling_loss_fibre": 0.,      # initial loss on channel to midpoint detectors
            "attenuation_l": 0,          # channel attenuation left of the detectors [dB/km]
            "attenuation_r": 0,          # channel attenuation right of the detectors [dB/km]
            "fibre_phase_stdv_l": .0,
            "fibre_phase_stdv_r": .0,
            # source
            "source_frequency": 20e6,                 # frequency of the photon pair source [Hz]
            "num_multiplexing_modes": 100,
            "mean_photon_number": None,               # mean photon pair number (only used if multi_photon=True)
            "emission_probabilities": [0., 1., 0., 0.],  # emission probs for photon pair sources
            # midpoint detector
            "det_dark_count_prob": 0,   # probability of dark count per detection
            "det_efficiency": 1.,       # detector efficiency
            "det_visibility": 1.,       # photon indistinguishability (must be 1 for multi_photon=True)
            "det_num_resolving": True,  # using number or non_number resolving detectors
            # swap & end node detector
            "swap_det_dark_count_prob": 0,   # probability of dark count per detection
            "swap_det_efficiency": 1.,       # detector efficiency
            "swap_det_visibility": 1.,       # photon indistinguishability (must be 1 for multi_photon=True)
            "swap_det_num_resolving": True,  # using number or non_number resolving detectors
            # memory
            "memory_coherence_time": 1e6,               # coherence time of the quantum memory [ns]
            "max_memory_efficiency": 1.,                # maximum efficiency of the quantum memory
            "memory_time_dependence": "exponential",    # time-dependence of efficiency
        }

In [ ]:
df = pd.read_csv('parameters_simulation.csv', index_col=None)
df.head()

In [ ]:
#array_column = 'emission_probabilities'

In [ ]:
'''
df = df.replace({np.nan: None})
sim_params = df.to_dict()

for key in list(sim_params.keys()):
    if not key==array_column:
        sim_params[key] = sim_params[key][0]
    else:
        sim_params[key] = list(df[key].values)
sim_params['attenuation_l'] = 0.0
sim_params['attenuation_r'] = 0.0    
sim_params
'''

In [211]:
class QSourceErrorModel(QuantumErrorModel):
    """Custom non-physical error model used to show the effectiveness
    of repeater chains.

    The default values are chosen to make a nice figure,
    and don't represent any physical system.

    Parameters
    ----------
    p_depol_init : float, optional
        Probability of depolarization on entering a fibre.
        Must be between 0 and 1. Default 0.009
    p_depol_length : float, optional
        Probability of depolarization per km of fibre.
        Must be between 0 and 1. Default 0.025

    """

    def __init__(self, p_error=0.1):
        super().__init__()
        self.properties['p_error'] = p_error

    def prob_item_lost(self, delta_time, **kwargs):
        """Uses the length property to calculate a depolarization probability,
        and applies it to the qubits.

        Parameters
        --------    sim_params["num_attempts"] = 10000
        qubits : tuple of :obj:`~netsquid.qubits.qubit.Qubit`
            Qubits to apply noise to.

        """
        return self.properties['p_error']
    
    def error_operation(self, qubits, delta_time, **kwargs):
        prob_loss = self.properties['p_error']
        for qubit in qubits:
            if get_random_state().random_sample() <= prob_loss:                
                if qubit.qstate is not None:
                        qapi.discard(qubit)


In [231]:
'''
    for node in network.nodes:
        if isinstance(network.subcomponents[node], EndNode):
            network.subcomponents[node].subcomponents['PPS'].models['emission_noise_model'] = QSourceErrorModel(p_error=0)
        if isinstance(network.subcomponents[node], RepeaterNode):
            network.subcomponents[node].subcomponents['PPS_L'].models['emission_noise_model'] = QSourceErrorModel(p_error=0)
            network.subcomponents[node].subcomponents['PPS_R'].models['emission_noise_model'] = QSourceErrorModel(p_error=0)
    for i, proto in enumerate(protocols):
        proto.start()
    for node in network.nodes.values():
        node.subcomponents["Clock"].start()

    link_state_collector = DataCollector(ChainStateCollector(nodes_1=list(network.nodes.values()))[:-2], include_time_stamp=True)
    ex_protos = [protocol for protocol in protocols if isinstance(protocol, ExtractionProtocol)]
    _state_collector.collect_on([(ex_protos[0], ExtractionProtocol.evtype_extract),
                                     (ex_protos[1], ExtractionProtocol.evtype_extract)], "AND")
    ns.sim_run(duration=10**9)
    return link_state_collector.dataframe
'''
"""Simulation script for an AE single repeater simulation using a full simulation or elementary link magic."""
import os
import time
import logging
import pickle5 as pickle
from matplotlib import rcParams
import matplotlib.pyplot as plt
from argparse import ArgumentParser

import netsquid as ns
from netsquid.util import DataCollector
from netsquid.util.simlog import logger

from pydynaa import EventExpression, ExpressionHandler

from netsquid_ae.ae_protocols import ExtractionProtocol
from netsquid_ae.protocol_event_types import EVTYPE_SUCCESS
from netsquid_ae.ae_chain_setup import create_single_repeater
from netsquid_ae.datacollectors import DecisionMaker, ChainStateCollector, MeasurementCollector

from netsquid_simulationtools.repchain_dataframe_holder import RepchainDataFrameHolder
from netsquid_simulationtools.repchain_data_process import process_repchain_dataframe_holder, process_data_bb84


def run_simulation(sim_params):
    """Set up and run a simulation for fixed set of parameters.

    Parameters
    ----------
    sim_params : dict
        Simulation parameters.

    Returns
    -------
    df_meas: instance of :class:`~pandas.DataFrame`
        DataFrame containing simulation/measurement results collected with the `measurement_collector`.
    df_state: instance of :class:`~pandas.DataFrame`
        DataFrame containing end-to-end state and relevant data collected with the `chain_collector`.

    """
    ns.sim_reset()

    # set up protocols and nodes (non-zero extraction_delay to be able to collect post-swap end-to-end state)
    protocols, network, _ = create_repeater_chain(**sim_params)
   # protocols, network, _ = create_single_repeater(extraction_delay=sim_params["time_step"]/10, **sim_params)
   
    det_protos = create_qkd_application(network=network, sim_params=sim_params, measure_directly=True,
                                        initial_measurement_basis="X")
    protocols.extend(det_protos)
    # Start Protocols
    for proto in protocols:
        print(proto)
        proto.start()
    # For Magic clocks don't matter, the Wizard triggers the protocols
    # Start Clocks
    for node in list(network.nodes.values())[:-2]:
        print(node)
        # Note: Detection nodes do not have clocks
        node.subcomponents["Clock"].start()
    source_error=0.0396
    for node in network.nodes:
        if isinstance(network.subcomponents[node], EndNode):
            network.subcomponents[node].subcomponents['PPS'].models['emission_noise_model'] = QSourceErrorModel(p_error=source_error)
        if isinstance(network.subcomponents[node], RepeaterNode):
            network.subcomponents[node].subcomponents['PPS_L'].models['emission_noise_model'] = QSourceErrorModel(p_error=source_error)
            network.subcomponents[node].subcomponents['PPS_R'].models['emission_noise_model'] = QSourceErrorModel(p_error=source_error)

    # Setup data collection
    # =====================
    # setting up EventExpressions for success at Alice and Bob (triggered by the MessagingProtocols at the detectors)
    print(protocols[-2], protocols[-1])
    evexpr_succ_a = EventExpression(event_type=EVTYPE_SUCCESS, source=protocols[-2])
    evexpr_succ_b = EventExpression(event_type=EVTYPE_SUCCESS, source=protocols[-1])
    evexpr_success = evexpr_succ_a and evexpr_succ_b

    # Using callable classes for data collection
    measurement_collector = DataCollector(MeasurementCollector(messaging_protocol_alice=protocols[-2],
                                                               messaging_protocol_bob=protocols[-1]), include_time_stamp=True)
#    chain_state_collector = DataCollector(ChainStateCollector(nodes_1=list(network.nodes.values())[:-2]))
    # decision maker (changes measurement basis and stops simulation)
    decision_maker = DecisionMaker(min_successes=sim_params["num_successes"], datacollector=measurement_collector,
                                   messaging_protocol_alice=protocols[-2],
                                   messaging_protocol_bob=protocols[-1])
    # set up Handler for EventExpression
    decision_handler = ExpressionHandler(decision_maker.decide)
    decision_maker._wait(decision_handler, expression=evexpr_success)
    # wait for EventExpression with DataCollector
    measurement_collector.collect_on(evexpr_success)
    # select the two extraction protocols for chain_state_collector
    ex_protos = [protocol for protocol in protocols if isinstance(protocol, ExtractionProtocol)]
 #   chain_state_collector.collect_on([(ex_protos[0], ExtractionProtocol.evtype_extract),
 #                                    (ex_protos[1], ExtractionProtocol.evtype_extract)], "AND")

    # Run simulation
    print("starting simulation with sim_parms", sim_params)
    # ns.logger.setLevel(logging.DEBUG)
    ns.sim_run()

    # filter chain_state_collector for non-empty lines
  #  state_data = chain_state_collector.dataframe

    return measurement_collector.dataframe# state_data

In [235]:


def run_simulation(sim_params):
    """Set up and run a simulation for fixed set of parameters.

    Parameters
    ----------
    sim_params : dict
        Simulation parameters.

    Returns
    -------
    df_meas: instance of :class:`~pandas.DataFrame`
        DataFrame containing simulation/measurement results collected with the `measurement_collector`.
    df_state: instance of :class:`~pandas.DataFrame`
        DataFrame containing end-to-end state and relevant data collected with the `chain_collector`.

    """
    ns.sim_reset()

    # set up protocols and nodes (non-zero extraction_delay to be able to collect post-swap end-to-end state)
    protocols, network, _ = create_elementary_link(**sim_params)
   # protocols, network, _ = create_single_repeater(extraction_delay=sim_params["time_step"]/10, **sim_params)
   
    for proto in protocols:
        proto.start()
    # For Magic clocks don't matter, the Wizard triggers the protocols
    # Start Clocks
    for node in list(network.nodes.values()):
        # Note: Detection nodes do not have clocks
        node.subcomponents["Clock"].start()
    source_error=0.0396
    for node in network.nodes:
        if isinstance(network.subcomponents[node], EndNode):
            network.subcomponents[node].subcomponents['PPS'].models['emission_noise_model'] = QSourceErrorModel(p_error=source_error)
        if isinstance(network.subcomponents[node], RepeaterNode):
            network.subcomponents[node].subcomponents['PPS_L'].models['emission_noise_model'] = QSourceErrorModel(p_error=source_error)
            network.subcomponents[node].subcomponents['PPS_R'].models['emission_noise_model'] = QSourceErrorModel(p_error=source_error)
    link_state_collector = DataCollector(ChainStateCollector(nodes_1=list(network.nodes.values())))
    link_state_collector.collect_on([(protocols[0], EVTYPE_SUCCESS), (protocols[1], EVTYPE_SUCCESS)], "AND")
    print("starting simulation with sim_parms", sim_params)
    ns.sim_run()

    # filter chain_state_collector for non-empty lines
  #  state_data = chain_state_collector.dataframe

    return link_state_collector.dataframe# state_data

In [237]:
 # Fixed simulation parameters
fixed_sim_params = {"num_repeaters": 1,         # number of repeaters
                        "encoding": "time_bin",     # encoding of the entangled photons
                        "temporal_modes": 100}        # number of temporal modes
    # Tunable simulation parameters
sim_params = {
        # SIMULATION
        "multi_photon": False,          # bool: whether to use multi-photon up to n=3 (WARNING: very slow)
        # end of simulation             Note: only use one and set others to -1
        "num_attempts": 1000,             # number of clicks after which the clock stops (thus stopping the protocols)
        "num_attempts_proto": -1,       # number of clicks after which the emission protocols stop
        "num_successes": -1,           # number of successes after which the DecisionMaker should stop the simulation
        # magic
        "magic": None,                  # whether we want to use magic (should be "analytical" or None)
        "state_files_directory": None,
        # clock                         Note: only used if magic = None, otherwise cycle time is used as time_step
        "time_step": 1e6,              # time step for the clock in ns , determines time between entanglement attempts
        # protocol
        "multiple_link_successes": True,  # whether the protocol uses multiple successful modes per elementary link

        # COMPONENTS
        # channel                       Note: use either length or node_distance and set the other one to -1
        "length": 60,                  # total distance between end node/ total channel length [km]
        "channel_length_l": 30,      # length of the channel left of the detectors [km]
        "channel_length_r": 30,      # length of the channel right of the detectors [km]
        "coupling_loss_fibre": 0.,      # initial loss on channel to the midpoint detectors
        "attenuation_l": 0.146,           # channel attenuation left of the detectors [dB/km]
        "attenuation_r": 0.146,           # channel attenuation right of the detectors [dB/km]
        "fibre_phase_stdv_l": .0,       # stdev of random phase picked up on the fibre in presence_absence encoding
        "fibre_phase_stdv_r": .0,       # stdev of random phase picked up on the fibre in presence_absence encoding
        # source
        "source_frequency": 10**9,       # frequency of the photon pair source [Hz]
        "num_multiplexing_modes": 300,           # number of multiplexing modes of the photon pair source
        "mean_photon_number": 0.05,     # mean photon pair number (only used if multi_photon=True)
        "emission_probabilities": [0, 1., 0, 0],   # emission probability for photon pair sources
        # detectors
        # midpoint
        "det_dark_count_prob": 0,    # probability of dark count per detection
        "det_efficiency": 0.99,          # detector efficiency
        "det_visibility": 1.,           # photon indistinguishability
        "det_num_resolving": False,     # using number or non_number resolving detectors
        # swap
        "swap_det_dark_count_prob": 0,   # probability of dark count per detection
        "swap_det_efficiency": 0.5,         # detector efficiency
        "swap_det_visibility": 1.,          # photon indistinguishability
        "swap_det_num_resolving": False,    # using number or non_number resolving detectors
        # memory
        "memory_coherence_time": 3*10**5,       # coherence time of the quantum memory [ns]
        "max_memory_efficiency": 0.75,       # maximum efficiency of the quantum memory
        "memory_time_dependence": "exponential"     # time-dependence of efficiency
    }

sim_params.update(fixed_sim_params)
df_meas = run_simulation(sim_params)

starting simulation with sim_parms {'multi_photon': False, 'num_attempts': 1000, 'num_attempts_proto': -1, 'num_successes': -1, 'magic': None, 'state_files_directory': None, 'time_step': 1000000.0, 'multiple_link_successes': True, 'length': 60, 'channel_length_l': 30, 'channel_length_r': 30, 'coupling_loss_fibre': 0.0, 'attenuation_l': 0.146, 'attenuation_r': 0.146, 'fibre_phase_stdv_l': 0.0, 'fibre_phase_stdv_r': 0.0, 'source_frequency': 1000000000, 'num_multiplexing_modes': 300, 'mean_photon_number': 0.05, 'emission_probabilities': [0, 1.0, 0, 0], 'det_dark_count_prob': 0, 'det_efficiency': 0.99, 'det_visibility': 1.0, 'det_num_resolving': False, 'swap_det_dark_count_prob': 0, 'swap_det_efficiency': 0.5, 'swap_det_visibility': 1.0, 'swap_det_num_resolving': False, 'memory_coherence_time': 300000, 'max_memory_efficiency': 0.75, 'memory_time_dependence': 'exponential', 'num_repeaters': 1, 'encoding': 'time_bin', 'temporal_modes': 100}
--No. of entanglement successes so far:  100 betwee

In [238]:
df_meas

,time_stamp,entity_name,state,number_of_rounds,round,midpoint_outcome_0
0,300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.5+0j), (0.5+0j), 0j...",1.0,1.0,1.0
1,1300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.49999999999999994+0...",1.0,2.0,2.0
2,2300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.5+0j), (-0.5+0j), 0...",1.0,3.0,2.0
3,3300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.5+0j), (-0.5+0j), 0...",1.0,4.0,2.0
4,4300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.49999999999999994+0...",1.0,5.0,1.0
...,...,...,...,...,...,...
904,994300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.5+0j), (-0.5+0j), 0...",1.0,995.0,2.0
905,995300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.5+0j), (-0.5+0j), 0...",1.0,996.0,2.0
906,996300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.5+0j), (0.5+0j), 0j...",1.0,997.0,1.0
907,997300000.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.49999999999999994+0...",1.0,998.0,2.0


In [240]:
257/(df_meas['time_stamp'].values[-1]/10**9)

257.43764399479113

In [241]:
times = np.diff(df_meas["time_stamp"].values/(10**9)).mean()
times**-1

909.8196392785572

In [139]:
not_drop, drop = 0,0
for i in range(10000000):
    if get_random_state().random_sample() <= 0.9:
        drop+=1
    else:
        not_drop+=1
drop/10000000

0.8999349

In [ ]:
# repeaters = 1
dict_dfs = {'df{}'.format(i): None for i in range(repeaters)}
for i in tqdm(range(repeaters)):
#    sim_params['num_attempts'] = 1000
#    sim_params[""]
#    sim_params["channel_length_l"] = sim_params["length"]/(2*(i+1)+2)
#    sim_params["channel_length_r"] = sim_params["length"]/(2*(i+1)+2)
#    sim_params["num_repeaters"] = i+1
    sim_params['num_attempts'] = 100000
    df, state = run_simulation(sim_params)
    print(df, state)
    #dict_dfs['df{}'.format(i)] = df
    #    print(len(df))    all_outcomes.append(BSMOutcome(success=False))
            
#    if len(df):
#        times = np.diff(df['time_stamp'].values/(10**9))    all_outcomes.append(node.cdata["swap_outcomes"][-1][0])
        
#        first_time = np.array([df['time_stamp'].values[0]/(10**9)])
#        times = np.append(first_time, times, axis = 0)
#        print(times.mean()**-1)

In [ ]:
dict_dfs['df0'].dropna()

In [ ]:
results = pd.read_csv('dist_rate.csv', index_col=None)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(results['num_links'].values, results['dist_rate'].values, '-v')
ax.set_yscale('log')
plt.grid()
plt.ylabel('Ent. Dist. Rate')
plt.xlabel('N.repeaters')
plt.show()